In [14]:
import os
import kagglehub
import tensorflow as tf

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

print("Libraries imported successfully.")

Libraries imported successfully.


In [9]:
print("Acquiring data sets...")

kagglehub.login()

path = kagglehub.dataset_download("sriramr/fruits-fresh-and-rotten-for-classification")

print(f"Data set was downloaded to {path}")

Acquiring data sets...


100%|██████████| 3.58G/3.58G [00:42<00:00, 90.7MB/s]

Extracting files...


Data set was downloaded to /root/.cache/kagglehub/datasets/sriramr/fruits-fresh-and-rotten-for-classification/versions/1
Kaggle credentials set.


In [10]:
print("Preparing data...")

# Const
BATCH_SIZE = 32         # maximum batch size
IMG_SIZE = (224, 224)   # maximum resolution of MobileNetV2

# Define paths
dataset_dir = os.path.join(path, "dataset")
train_dir = os.path.join(dataset_dir, "train")
test_dir = os.path.join(dataset_dir, "test")

# Normalize the pixel colors from 1 to 255
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range = 20,
    horizontal_flip=True
)

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    test_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)

validation_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)


# labels
labels = list(train_generator.class_indices.keys())
print(f"\n✅ Detected classes: {labels}")

# Save to file
with open("labels.txt", "w") as f:
  for label in labels:
    f.write(label + "\n")


Preparing data...
Found 2698 images belonging to 6 classes.
Found 2698 images belonging to 6 classes.

✅ Detected classes: ['freshapples', 'freshbanana', 'freshoranges', 'rottenapples', 'rottenbanana', 'rottenoranges']


In [12]:
print("Building the MobileNetV2 model...")

# Cut off head
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# keep previous memory of the model
base_model.trainable = False

# Add custom head
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.2)(x)

# Turn raw numbers into probabilities
predictions = Dense(train_generator.num_classes, activation='softmax')(x)

# merge them
model = Model(inputs=base_model.input, outputs=predictions)


# Compile
model.compile(
    optimizer=Adam(learning_rate=0.0001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
    )

print("Modela architecture built successfully...")

Building the MobileNetV2 model...
Modela architecture built successfully...


In [13]:
from IPython.core import history
print("Model training process was started... (approx. 5 - 10 mins)")


# Fit the model
history = model.fit(
    train_generator,
    epochs=5,
    validation_data=validation_generator
)

Model training process was started... (approx. 5 - 10 mins)


/usr/local/lib/python3.12/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/5
85/85 ━━━━━━━━━━━━━━━━━━━━ 304s 3s/step - accuracy: 0.2866 - loss: 1.9029 - val_accuracy: 0.5990 - val_loss: 1.1300
Epoch 2/5
85/85 ━━━━━━━━━━━━━━━━━━━━ 285s 3s/step - accuracy: 0.6067 - loss: 1.0598 - val_accuracy: 0.8076 - val_loss: 0.7443
Epoch 3/5
85/85 ━━━━━━━━━━━━━━━━━━━━ 269s 3s/step - accuracy: 0.7588 - loss: 0.7628 - val_accuracy: 0.8673 - val_loss: 0.5569
Epoch 4/5
85/85 ━━━━━━━━━━━━━━━━━━━━ 335s 3s/step - accuracy: 0.8396 - loss: 0.5684 - val_accuracy: 0.8895 - val_loss: 0.4531
Epoch 5/5
85/85 ━━━━━━━━━━━━━━━━━━━━ 272s 3s/step - accuracy: 0.8657 - loss: 0.4892 - val_accuracy: 0.9062 - val_loss: 0.3804


In [17]:
print ("Converting the trained model into .TFLite file...")

# Create converter
converter = tf.lite.TFLiteConverter.from_keras_model(model)

# Convert
tflite_model = converter.convert()

# Save
with open("Freshio_model_v01.tflite", 'wb') as f:
  f.write(tflite_model)

print("Freshio model version 01 was successfully trained...")

Converting the trained model into .TFLite file...
Saved artifact at '/tmp/tmprv2injo_'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name='keras_tensor_154')
Output Type:
  TensorSpec(shape=(None, 6), dtype=tf.float32, name=None)
Captures:
  137443318551504: TensorSpec(shape=(), dtype=tf.resource, name=None)
  137443318553040: TensorSpec(shape=(), dtype=tf.resource, name=None)
  137443318552848: TensorSpec(shape=(), dtype=tf.resource, name=None)
  137443318551696: TensorSpec(shape=(), dtype=tf.resource, name=None)
  137443318553616: TensorSpec(shape=(), dtype=tf.resource, name=None)
  137443318549200: TensorSpec(shape=(), dtype=tf.resource, name=None)
  137443318553232: TensorSpec(shape=(), dtype=tf.resource, name=None)
  137443318553424: TensorSpec(shape=(), dtype=tf.resource, name=None)
  137443318552464: TensorSpec(shape=(), dtype=tf.resource, name=None)
  137443318554576: TensorSpec(sh